In [1]:
import numpy as np
import SequenceGen
import URI_tl
import URI_ntl
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import spikeship.spikeship as spikeship
from scipy.stats import ttest_ind

C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numba\core\decorators.py:282: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
C:\Users\tomha\OneDrive\Bureaublad\BSC Thesis\Analysis\spikeship\spikeship.py:254: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(int32(int32, int32))


In [2]:
def convert_spike_timing(spikes_neuron_trial):
    nr_trials = spikes_neuron_trial.shape[0]
    nr_neurons = spikes_neuron_trial.shape[1]
    ii_spike_times = np.empty((nr_trials, nr_neurons, 2))
    spike_times = []
    idx = 0
    for trial in range(nr_trials):
        for channel in range(nr_neurons):
            spikes = spikes_neuron_trial[trial, channel]
            spike_times.extend(spikes)
            ii_spike_times[trial, channel, 0] = idx
            idx += len(spikes)
            ii_spike_times[trial, channel, 1] = idx
    spike_times = np.array(spike_times)
    return spike_times, ii_spike_times

# Time-locked

In [3]:
# Parameters for experiment

# To add?
# Probability of having a sequence (<=1)
# Probability of detecting a spike in the sequence (<=1) a.k.a. partial sequence

# Constant
nr_neurons = 50
FR_var = 0. # Spikeship ttest does not work if FR_var > 0 
trial_len = 0.1
seq_len = 0.04
mean_delay = 0.03 # (timing of sequence relative to trial)

# Variable
nr_trials = np.arange(10, 300)
neurons_in_seq = np.arange(2, 15)
jitter = np.linspace(0, 0.05, 100)
delay_var = np.linspace(0, 0.03, 100)
avg_FR = np.linspace(2, 25, 100)

# Default parameters
nr_trials_ = 100
neurons_in_seq_ = 5
jitter_ = 0.01
delay_var_ = 0.01
avg_FR_ = 10

Interactions:
- nr_trials with noise
- nr_trials with neurons_in_seq
- noise with neurons_in_seq (might be hard to motivate)

In [4]:
# Experiment type A

test_params = jitter
# Empty arrays with one position for each experiment
URI_scores = np.empty(test_params.shape[0])
p_vals = np.empty(test_params.shape[0])
surr_avgs = np.empty(test_params.shape[0])
surr_95s = np.empty(test_params.shape[0])
URI_scores_ = np.empty(test_params.shape[0])
p_vals_ = np.empty(test_params.shape[0])
surr_avgs_ = np.empty(test_params.shape[0])
surr_95s_ = np.empty(test_params.shape[0])
spikeship_tscores = np.empty(test_params.shape[0])
spikeship_pvals = np.empty(test_params.shape[0])

for idx in tqdm(range(len(test_params))):
    val = test_params[idx]
    # Generate the data
    data_seq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                      avg_FR=avg_FR_, FR_var=FR_var, has_sequence=True, jitter=val, nr_in_sequence=neurons_in_seq_)
    data_noseq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                      avg_FR=avg_FR_, FR_var=FR_var, has_sequence=False, jitter=val, nr_in_sequence=neurons_in_seq_)
    # Compute URI score on data with and without a sequence
    URI, p_val, occ, surr_avg, surr_95 = URI_tl.getURI(data_seq, 500)
    URI_scores[idx] = URI
    p_vals[idx] = p_val
    surr_avgs[idx] = surr_avg
    surr_95s[idx] = surr_95
    URI_, p_val_, occ_, surr_avg_, surr_95_ = URI_tl.getURI(data_noseq, 500)
    URI_scores_[idx] = URI_
    p_vals_[idx] = p_val_
    surr_avgs_[idx] = surr_avg_
    surr_95s_[idx] = surr_95_
    # Get internal SpikeShip distances for data with and without a sequence and perform two sample t-test
    data_seq_convert, conversion1 = convert_spike_timing(data_seq)
    data_noseq_convert, conversion2 = convert_spike_timing(data_noseq)
    distances_seq = spikeship.distances(data_seq_convert, conversion1)
    distances_noseq = spikeship.distances(data_noseq_convert, conversion2)
    distances_seq = distances_seq[distances_seq != 0]
    distances_noseq = distances_noseq[distances_noseq != 0]
    res = ttest_ind(distances_seq.flatten(), data_noseq_convert.flatten(), alternative='less')
    spikeship_tscores[idx] = res.statistic
    spikeship_pvals[idx] = res.pvalue

np.save("Results/jitter/URI_scores", URI_scores)
np.save("Results/jitter/p_vals", p_vals)
np.save("Results/jitter/surr_avgs", surr_avgs)
np.save("Results/jitter/surr_95s", surr_95s)
np.save("Results/jitter/URI_scores_", URI_scores_)
np.save("Results/jitter/p_vals_", p_vals_)
np.save("Results/jitter/surr_avgs_", surr_avgs_)
np.save("Results/jitter/surr_95s_", surr_95s_)
np.save("Results/jitter/spikeship_tscores", spikeship_tscores)
np.save("Results/jitter/spikeship_pvals", spikeship_pvals)

  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\lib\function_base.py:2412: RuntimeWarning: invalid value encountered in mean (vectorized)
  outputs = ufunc(*inputs)


In [7]:
# Experiment type A

test_params = nr_trials
# Empty arrays with one position for each experiment
URI_scores = np.empty(test_params.shape[0])
p_vals = np.empty(test_params.shape[0])
surr_avgs = np.empty(test_params.shape[0])
surr_95s = np.empty(test_params.shape[0])
URI_scores_ = np.empty(test_params.shape[0])
p_vals_ = np.empty(test_params.shape[0])
surr_avgs_ = np.empty(test_params.shape[0])
surr_95s_ = np.empty(test_params.shape[0])
spikeship_tscores = np.empty(test_params.shape[0])
spikeship_pvals = np.empty(test_params.shape[0])

for idx in tqdm(range(len(test_params))):
    val = test_params[idx]
    # Generate the data
    data_seq, _ = SequenceGen.simulate_trials(val, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                      avg_FR=avg_FR_, FR_var=FR_var, has_sequence=True, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
    data_noseq, _ = SequenceGen.simulate_trials(val, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                      avg_FR=avg_FR_, FR_var=FR_var, has_sequence=False, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
    # Compute URI score on data with and without a sequence
    URI, p_val, occ, surr_avg, surr_95 = URI_tl.getURI(data_seq, 500)
    URI_scores[idx] = URI
    p_vals[idx] = p_val
    surr_avgs[idx] = surr_avg
    surr_95s[idx] = surr_95
    URI_, p_val_, occ_, surr_avg_, surr_95_ = URI_tl.getURI(data_noseq, 500)
    URI_scores_[idx] = URI_
    p_vals_[idx] = p_val_
    surr_avgs_[idx] = surr_avg_
    surr_95s_[idx] = surr_95_
    # Get internal SpikeShip distances for data with and without a sequence and perform two sample t-test
    data_seq_convert, conversion1 = convert_spike_timing(data_seq)
    data_noseq_convert, conversion2 = convert_spike_timing(data_noseq)
    distances_seq = spikeship.distances(data_seq_convert, conversion1)
    distances_noseq = spikeship.distances(data_noseq_convert, conversion2)
    distances_seq = distances_seq[distances_seq != 0]
    distances_noseq = distances_noseq[distances_noseq != 0]
    res = ttest_ind(distances_seq.flatten(), data_noseq_convert.flatten(), alternative='less')
    spikeship_tscores[idx] = res.statistic
    spikeship_pvals[idx] = res.pvalue

np.save("Results/nr_trials/URI_scores", URI_scores)
np.save("Results/nr_trials/p_vals", p_vals)
np.save("Results/nr_trials/surr_avgs", surr_avgs)
np.save("Results/nr_trials/surr_95s", surr_95s)
np.save("Results/nr_trials/URI_scores_", URI_scores_)
np.save("Results/nr_trials/p_vals_", p_vals_)
np.save("Results/nr_trials/surr_avgs_", surr_avgs_)
np.save("Results/nr_trials/surr_95s_", surr_95s_)
np.save("Results/nr_trials/spikeship_tscores", spikeship_tscores)
np.save("Results/nr_trials/spikeship_pvals", spikeship_pvals)

  0%|          | 0/290 [00:00<?, ?it/s]

In [7]:
# Experiment type A

test_params = neurons_in_seq
nr_exp = 5
# Empty arrays with one position for each experiment
URI_scores = np.empty((test_params.shape[0], nr_exp))
p_vals = np.empty((test_params.shape[0], nr_exp))
surr_avgs = np.empty((test_params.shape[0], nr_exp))
surr_95s = np.empty((test_params.shape[0], nr_exp))
URI_scores_ = np.empty((test_params.shape[0], nr_exp))
p_vals_ = np.empty((test_params.shape[0], nr_exp))
surr_avgs_ = np.empty((test_params.shape[0], nr_exp))
surr_95s_ = np.empty((test_params.shape[0], nr_exp))
spikeship_tscores = np.empty((test_params.shape[0], nr_exp))
spikeship_pvals = np.empty((test_params.shape[0], nr_exp))

for idx in tqdm(range(len(test_params))):
    for exp_nr in range(nr_exp):
        val = test_params[idx]
        # Generate the data
        data_seq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                          avg_FR=avg_FR_, FR_var=FR_var, has_sequence=True, jitter=jitter_, nr_in_sequence=val)
        data_noseq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                          avg_FR=avg_FR_, FR_var=FR_var, has_sequence=False, jitter=jitter_, nr_in_sequence=val)
        # Compute URI score on data with and without a sequence
        URI, p_val, occ, surr_avg, surr_95 = URI_tl.getURI(data_seq, 500)
        URI_scores[idx, exp_nr] = URI
        p_vals[idx, exp_nr] = p_val
        surr_avgs[idx, exp_nr] = surr_avg
        surr_95s[idx, exp_nr] = surr_95
        URI_, p_val_, occ_, surr_avg_, surr_95_ = URI_tl.getURI(data_noseq, 500)
        URI_scores_[idx, exp_nr] = URI_
        p_vals_[idx, exp_nr] = p_val_
        surr_avgs_[idx, exp_nr] = surr_avg_
        surr_95s_[idx, exp_nr] = surr_95_
        # Get internal SpikeShip distances for data with and without a sequence and perform two sample t-test
        data_seq_convert, conversion1 = convert_spike_timing(data_seq)
        data_noseq_convert, conversion2 = convert_spike_timing(data_noseq)
        distances_seq = spikeship.distances(data_seq_convert, conversion1)
        distances_noseq = spikeship.distances(data_noseq_convert, conversion2)
        distances_seq = distances_seq[distances_seq != 0]
        distances_noseq = distances_noseq[distances_noseq != 0]
        res = ttest_ind(distances_seq.flatten(), data_noseq_convert.flatten(), alternative='less')
        spikeship_tscores[idx, exp_nr] = res.statistic
        spikeship_pvals[idx, exp_nr] = res.pvalue

np.save("Results/neurons_in_seq/URI_scores", URI_scores)
np.save("Results/neurons_in_seq/p_vals", p_vals)
np.save("Results/neurons_in_seq/surr_avgs", surr_avgs)
np.save("Results/neurons_in_seq/surr_95s", surr_95s)
np.save("Results/neurons_in_seq/URI_scores_", URI_scores_)
np.save("Results/neurons_in_seq/p_vals_", p_vals_)
np.save("Results/neurons_in_seq/surr_avgs_", surr_avgs_)
np.save("Results/neurons_in_seq/surr_95s_", surr_95s_)
np.save("Results/neurons_in_seq/spikeship_tscores", spikeship_tscores)
np.save("Results/neurons_in_seq/spikeship_pvals", spikeship_pvals)

  0%|          | 0/13 [00:00<?, ?it/s]

In [4]:
# Experiment type A

test_params = delay_var
# Empty arrays with one position for each experiment
URI_scores = np.empty(test_params.shape[0])
p_vals = np.empty(test_params.shape[0])
surr_avgs = np.empty(test_params.shape[0])
surr_95s = np.empty(test_params.shape[0])
URI_scores_ = np.empty(test_params.shape[0])
p_vals_ = np.empty(test_params.shape[0])
surr_avgs_ = np.empty(test_params.shape[0])
surr_95s_ = np.empty(test_params.shape[0])
spikeship_tscores = np.empty(test_params.shape[0])
spikeship_pvals = np.empty(test_params.shape[0])

for idx in tqdm(range(len(test_params))):
    val = test_params[idx]
    # Generate the data
    data_seq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=val,
                                      avg_FR=avg_FR_, FR_var=FR_var, has_sequence=True, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
    data_noseq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=val,
                                      avg_FR=avg_FR_, FR_var=FR_var, has_sequence=False, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
    # Compute URI score on data with and without a sequence
    URI, p_val, occ, surr_avg, surr_95 = URI_tl.getURI(data_seq, 500)
    URI_scores[idx] = URI
    p_vals[idx] = p_val
    surr_avgs[idx] = surr_avg
    surr_95s[idx] = surr_95
    URI_, p_val_, occ_, surr_avg_, surr_95_ = URI_tl.getURI(data_noseq, 500)
    URI_scores_[idx] = URI_
    p_vals_[idx] = p_val_
    surr_avgs_[idx] = surr_avg_
    surr_95s_[idx] = surr_95_
    # Get internal SpikeShip distances for data with and without a sequence and perform two sample t-test
    data_seq_convert, conversion1 = convert_spike_timing(data_seq)
    data_noseq_convert, conversion2 = convert_spike_timing(data_noseq)
    distances_seq = spikeship.distances(data_seq_convert, conversion1)
    distances_noseq = spikeship.distances(data_noseq_convert, conversion2)
    distances_seq = distances_seq[distances_seq != 0]
    distances_noseq = distances_noseq[distances_noseq != 0]
    res = ttest_ind(distances_seq.flatten(), data_noseq_convert.flatten(), alternative='less')
    spikeship_tscores[idx] = res.statistic
    spikeship_pvals[idx] = res.pvalue

np.save("Results/delay_var/URI_scores", URI_scores)
np.save("Results/delay_var/p_vals", p_vals)
np.save("Results/delay_var/surr_avgs", surr_avgs)
np.save("Results/delay_var/surr_95s", surr_95s)
np.save("Results/delay_var/URI_scores_", URI_scores_)
np.save("Results/delay_var/p_vals_", p_vals_)
np.save("Results/delay_var/surr_avgs_", surr_avgs_)
np.save("Results/delay_var/surr_95s_", surr_95s_)
np.save("Results/delay_var/spikeship_tscores", spikeship_tscores)
np.save("Results/delay_var/spikeship_pvals", spikeship_pvals)

  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\lib\function_base.py:2412: RuntimeWarning: invalid value encountered in mean (vectorized)
  outputs = ufunc(*inputs)


In [9]:
# Experiment type A

test_params = avg_FR
# Empty arrays with one position for each experiment
URI_scores = np.empty(test_params.shape[0])
p_vals = np.empty(test_params.shape[0])
surr_avgs = np.empty(test_params.shape[0])
surr_95s = np.empty(test_params.shape[0])
URI_scores_ = np.empty(test_params.shape[0])
p_vals_ = np.empty(test_params.shape[0])
surr_avgs_ = np.empty(test_params.shape[0])
surr_95s_ = np.empty(test_params.shape[0])
spikeship_tscores = np.empty(test_params.shape[0])
spikeship_pvals = np.empty(test_params.shape[0])

for idx in tqdm(range(len(test_params))):
    val = test_params[idx]
    # Generate the data
    data_seq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                      avg_FR=val, FR_var=FR_var, has_sequence=True, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
    data_noseq, _ = SequenceGen.simulate_trials(nr_trials_, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                      avg_FR=val, FR_var=FR_var, has_sequence=False, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
    # Compute URI score on data with and without a sequence
    URI, p_val, occ, surr_avg, surr_95 = URI_tl.getURI(data_seq, 500)
    URI_scores[idx] = URI
    p_vals[idx] = p_val
    surr_avgs[idx] = surr_avg
    surr_95s[idx] = surr_95
    URI_, p_val_, occ_, surr_avg_, surr_95_ = URI_tl.getURI(data_noseq, 500)
    URI_scores_[idx] = URI_
    p_vals_[idx] = p_val_
    surr_avgs_[idx] = surr_avg_
    surr_95s_[idx] = surr_95_
    # Get internal SpikeShip distances for data with and without a sequence and perform two sample t-test
    data_seq_convert, conversion1 = convert_spike_timing(data_seq)
    data_noseq_convert, conversion2 = convert_spike_timing(data_noseq)
    distances_seq = spikeship.distances(data_seq_convert, conversion1)
    distances_noseq = spikeship.distances(data_noseq_convert, conversion2)
    distances_seq = distances_seq[distances_seq != 0]
    distances_noseq = distances_noseq[distances_noseq != 0]
    res = ttest_ind(distances_seq.flatten(), data_noseq_convert.flatten(), alternative='less')
    spikeship_tscores[idx] = res.statistic
    spikeship_pvals[idx] = res.pvalue

np.save("Results/avg_FR/URI_scores", URI_scores)
np.save("Results/avg_FR/p_vals", p_vals)
np.save("Results/avg_FR/surr_avgs", surr_avgs)
np.save("Results/avg_FR/surr_95s", surr_95s)
np.save("Results/avg_FR/URI_scores_", URI_scores_)
np.save("Results/avg_FR/p_vals_", p_vals_)
np.save("Results/avg_FR/surr_avgs_", surr_avgs_)
np.save("Results/avg_FR/surr_95s_", surr_95s_)
np.save("Results/avg_FR/spikeship_tscores", spikeship_tscores)
np.save("Results/avg_FR/spikeship_pvals", spikeship_pvals)

  0%|          | 0/100 [00:00<?, ?it/s]

# Interaction effect

In [4]:
nr_trials_ie = np.arange(20, 320, 20)
avg_FR_ie = np.linspace(2, 25, 15)

In [7]:
# Experiment type A


# Empty arrays with one position for each experiment
URI_scores        = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
p_vals            = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
surr_avgs         = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
surr_95s          = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
URI_scores_       = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
p_vals_           = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
surr_avgs_        = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
surr_95s_         = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
spikeship_tscores = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))
spikeship_pvals   = np.empty((nr_trials_ie.shape[0], avg_FR_ie.shape[0]))

for idx_i in tqdm(range(len(nr_trials_ie))):
    for idx_j in range(len(avg_FR_ie)):
        nr_trials_i = nr_trials_ie[idx_i]
        avg_FR_j = avg_FR_ie[idx_j]
        # Generate the data
        data_seq, _ = SequenceGen.simulate_trials(nr_trials_i, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                          avg_FR=avg_FR_j, FR_var=FR_var, has_sequence=True, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
        data_noseq, _ = SequenceGen.simulate_trials(nr_trials_i, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                          avg_FR=avg_FR_j, FR_var=FR_var, has_sequence=False, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
        # Compute URI score on data with and without a sequence
        URI, p_val, occ, surr_avg, surr_95 = URI_tl.getURI(data_seq, 500)
        URI_scores[idx_i, idx_j] = URI
        p_vals[idx_i, idx_j] = p_val
        surr_avgs[idx_i, idx_j] = surr_avg
        surr_95s[idx_i, idx_j] = surr_95
        URI_, p_val_, occ_, surr_avg_, surr_95_ = URI_tl.getURI(data_noseq, 500)
        URI_scores_[idx_i, idx_j] = URI_
        p_vals_[idx_i, idx_j] = p_val_
        surr_avgs_[idx_i, idx_j] = surr_avg_
        surr_95s_[idx_i, idx_j] = surr_95_
        # Get internal SpikeShip distances for data with and without a sequence and perform two sample t-test
        data_seq_convert, conversion1 = convert_spike_timing(data_seq)
        data_noseq_convert, conversion2 = convert_spike_timing(data_noseq)
        distances_seq = spikeship.distances(data_seq_convert, conversion1)
        distances_noseq = spikeship.distances(data_noseq_convert, conversion2)
        distances_seq = distances_seq[distances_seq != 0]
        distances_noseq = distances_noseq[distances_noseq != 0]
        res = ttest_ind(distances_seq.flatten(), data_noseq_convert.flatten(), alternative='less')
        spikeship_tscores[idx_i, idx_j] = res.statistic
        spikeship_pvals[idx_i, idx_j] = res.pvalue

np.save("Results/interaction1/URI_scores", URI_scores)
np.save("Results/interaction1/p_vals", p_vals)
np.save("Results/interaction1/surr_avgs", surr_avgs)
np.save("Results/interaction1/surr_95s", surr_95s)
np.save("Results/interaction1/URI_scores_", URI_scores_)
np.save("Results/interaction1/p_vals_", p_vals_)
np.save("Results/interaction1/surr_avgs_", surr_avgs_)
np.save("Results/interaction1/surr_95s_", surr_95s_)
np.save("Results/interaction1/spikeship_tscores", spikeship_tscores)
np.save("Results/interaction1/spikeship_pvals", spikeship_pvals)

  0%|          | 0/15 [00:00<?, ?it/s]

C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\lib\function_base.py:2412: RuntimeWarning: invalid value encountered in mean (vectorized)
  outputs = ufunc(*inputs)


In [8]:
nr_trials_ie = np.arange(20, 320, 20)
neurons_in_seq_ie = np.arange(2, 17)

In [9]:
# Experiment type A


# Empty arrays with one position for each experiment
URI_scores        = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
p_vals            = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
surr_avgs         = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
surr_95s          = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
URI_scores_       = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
p_vals_           = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
surr_avgs_        = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
surr_95s_         = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
spikeship_tscores = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))
spikeship_pvals   = np.empty((nr_trials_ie.shape[0], neurons_in_seq_ie.shape[0]))

for idx_i in tqdm(range(len(nr_trials_ie))):
    for idx_j in range(len(neurons_in_seq_ie)):
        nr_trials_i = nr_trials_ie[idx_i]
        neurons_in_seq_j = neurons_in_seq_ie[idx_j]
        # Generate the data
        data_seq, _ = SequenceGen.simulate_trials(nr_trials_i, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                          avg_FR=avg_FR_, FR_var=FR_var, has_sequence=True, jitter=jitter_, nr_in_sequence=neurons_in_seq_j)
        data_noseq, _ = SequenceGen.simulate_trials(nr_trials_i, nr_neurons, trial_len, seq_len, mean_delay=mean_delay, delay_var=delay_var_,
                                          avg_FR=avg_FR_, FR_var=FR_var, has_sequence=False, jitter=jitter_, nr_in_sequence=neurons_in_seq_j)
        # Compute URI score on data with and without a sequence
        URI, p_val, occ, surr_avg, surr_95 = URI_tl.getURI(data_seq, 500)
        URI_scores[idx_i, idx_j] = URI
        p_vals[idx_i, idx_j] = p_val
        surr_avgs[idx_i, idx_j] = surr_avg
        surr_95s[idx_i, idx_j] = surr_95
        URI_, p_val_, occ_, surr_avg_, surr_95_ = URI_tl.getURI(data_noseq, 500)
        URI_scores_[idx_i, idx_j] = URI_
        p_vals_[idx_i, idx_j] = p_val_
        surr_avgs_[idx_i, idx_j] = surr_avg_
        surr_95s_[idx_i, idx_j] = surr_95_
        # Get internal SpikeShip distances for data with and without a sequence and perform two sample t-test
        data_seq_convert, conversion1 = convert_spike_timing(data_seq)
        data_noseq_convert, conversion2 = convert_spike_timing(data_noseq)
        distances_seq = spikeship.distances(data_seq_convert, conversion1)
        distances_noseq = spikeship.distances(data_noseq_convert, conversion2)
        distances_seq = distances_seq[distances_seq != 0]
        distances_noseq = distances_noseq[distances_noseq != 0]
        res = ttest_ind(distances_seq.flatten(), data_noseq_convert.flatten(), alternative='less')
        spikeship_tscores[idx_i, idx_j] = res.statistic
        spikeship_pvals[idx_i, idx_j] = res.pvalue

np.save("Results/interaction2/URI_scores", URI_scores)
np.save("Results/interaction2/p_vals", p_vals)
np.save("Results/interaction2/surr_avgs", surr_avgs)
np.save("Results/interaction2/surr_95s", surr_95s)
np.save("Results/interaction2/URI_scores_", URI_scores_)
np.save("Results/interaction2/p_vals_", p_vals_)
np.save("Results/interaction2/surr_avgs_", surr_avgs_)
np.save("Results/interaction2/surr_95s_", surr_95s_)
np.save("Results/interaction2/spikeship_tscores", spikeship_tscores)
np.save("Results/interaction2/spikeship_pvals", spikeship_pvals)

  0%|          | 0/15 [00:00<?, ?it/s]

C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\tomha\anaconda3\envs\BSC\lib\site-packages\numpy\lib\function_base.py:2412: RuntimeWarning: invalid value encountered in mean (vectorized)
  outputs = ufunc(*inputs)


# Non-time-locked

In [16]:
noise_lvl = 1
nr_trials_NTL = 5
trial_len_NTL = 100
data_seq, _, participating = SequenceGen.simulate_unaligned(nr_trials_NTL, nr_neurons, trial_len_NTL, avg_FR=noise_lvl, FR_var=FR_var,
                       time_btwn_seq=2, seq_length=seq_len, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
data_noseq, _, _ = SequenceGen.simulate_unaligned(nr_trials_NTL, nr_neurons, trial_len_NTL, avg_FR=noise_lvl, FR_var=FR_var,
                       time_btwn_seq=5, seq_length=seq_len, jitter=jitter_, nr_in_sequence=0)
URIs, URI_pvals = URI_ntl.get_URI_nontimelocked(data_seq, 500, window=(-0.1, 0.1))
URIs_, URI_pvals_ = URI_ntl.get_URI_nontimelocked(data_noseq, 500, window=(-0.1, 0.1))

np.save("Results/NTL/URIs1", URIs)
np.save("Results/NTL/URI_pvals1", URI_pvals)
np.save("Results/NTL/participating1", participating)
np.save("Results/NTL/URIs_1", URIs_)
np.save("Results/NTL/URI_pvals_1", URI_pvals_)

In [17]:
noise_lvl = 5
nr_trials_NTL = 5
trial_len_NTL = 100
data_seq, _, participating = SequenceGen.simulate_unaligned(nr_trials_NTL, nr_neurons, trial_len_NTL, avg_FR=noise_lvl, FR_var=FR_var,
                       time_btwn_seq=2, seq_length=seq_len, jitter=jitter_, nr_in_sequence=neurons_in_seq_)
data_noseq, _, _ = SequenceGen.simulate_unaligned(nr_trials_NTL, nr_neurons, trial_len_NTL, avg_FR=noise_lvl, FR_var=FR_var,
                       time_btwn_seq=5, seq_length=seq_len, jitter=jitter_, nr_in_sequence=0)
URIs, URI_pvals = URI_ntl.get_URI_nontimelocked(data_seq, 500, window=(-0.1, 0.1))
URIs_, URI_pvals_ = URI_ntl.get_URI_nontimelocked(data_noseq, 500, window=(-0.1, 0.1))

np.save("Results/NTL/URIs5", URIs)
np.save("Results/NTL/URI_pvals5", URI_pvals)
np.save("Results/NTL/participating5", participating)
np.save("Results/NTL/URIs_5", URIs_)
np.save("Results/NTL/URI_pvals_5", URI_pvals_)